# Colab Demo
This notebook performs downloads preprocessed data from S3, as well as a prefit pipeline which can be used to demo predictions. Additionally, ~16M pre-predicted properties are downloaded such that results can be explored. 


<p align="center">
<img width=25% src="https://blockchainclimate.org/wp-content/uploads/2020/11/cropped-BCI_Logo_LR-400x333.png" alt="bciAVM" height="300"/>
</p>

[![PyPI](https://badge.fury.io/py/bciavm.svg?maxAge=2592000)](https://badge.fury.io/py/bciavm)
[![PyPI Stats](https://img.shields.io/badge/bciavm-avm-blue)](https://pypistats.org/packages/bciavm)


This notebook contains code to take a `mlflow` registered model and distribute its work with a `Dask` cluster. 
<table>
    <tr>
        <td>
            <img width=25% src="https://saturn-public-assets.s3.us-east-2.amazonaws.com/example-resources/dask.png" width="300">
        </td>
    </tr>
</table>

The Blockchain & Climate Institute (BCI) is a progressive think tank providing leading expertise in the deployment of emerging technologies for climate and sustainability actions. 

As an international network of scientific and technological experts, BCI is at the forefront of innovative efforts, enabling technology transfers, to create a sustainable and clean global future.

# Automated Valuation Model (AVM) 

### About
AVM is a term for a service that uses mathematical modeling combined with databases of existing properties and transactions to calculate real estate values. 
The majority of automated valuation models (AVMs) compare the values of similar properties at the same point in time. 
Many appraisers, and even Wall Street institutions, use this type of model to value residential properties. (see [What is an AVM](https://www.investopedia.com/terms/a/automated-valuation-model.asp) Investopedia.com)

For more detailed info about the AVM, please read the **About** paper found here `resources/2021-BCI-AVM-About.pdf`.

### Valuation Process
<img src="resources/valuation_process.png" height="360" >


### Next Steps
Read more about bciAVM on our [documentation page](https://blockchainclimate.org/thought-leadership/#blog):

### How does it relate to BCI Risk Modeling?
<img src="resources/bci_flowchart_2.png" height="280" >


### Technical & financial support for development provided by:
<a href="https://www.gcode.ai">
    <img width=15% src="https://staticfiles-img.s3.amazonaws.com/avm/gcode_logo.png" alt="GCODE.ai"  height="25"/>
</a>


### Install [from PyPI](https://pypi.org/project/bciavm/)
```shell
pip install bciavm
```

This notebook covers the following steps:
- Download test data from s3
- Use a prefit pipeline to call predict on the test dataset
- Explore results of AVM

<hr>

## Environment Setup

In addition to the `bciavm` package, install the following additional non-builtin libraries:

* pandas_profiling

In [3]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
BUCKET_NAME = 'bci-public' 
pipeline_path = 'avm_pipeline_1.21.25.pkl'
X_test_path = 'X_test.parquet.gzip'
y_test_path = 'y_test.parquet.gzip'
PATHS = [pipeline_path, X_test_path, y_test_path]
for _path in PATHS:
  s3.Bucket(BUCKET_NAME).download_file(_path, _path)


In [4]:
import bciavm
import pandas as pd

pipeline = bciavm.pipelines.RegressionPipeline.load(pipeline_path)
X_test = pd.read_parquet(X_test_path)
y_test = pd.read_parquet(y_test_path)

In [5]:
input_example=preprocessed_data.dropna().sample(1)
input_example

Out[8]:

,unit_indx,POSTCODE,POSTCODE_OUTCODE,POSTCODE_AREA,POSTTOWN_e,PROPERTY_TYPE_e,TOTAL_FLOOR_AREA_e,NUMBER_HEATED_ROOMS_e,FLOOR_LEVEL_e,Latitude_m,Longitude_m
15289924,15289924,SW10 0TU,SW10,SW,LONDON,Flat,34.0,1.0,2.0,51.480952,-0.183493


In [6]:
pipeline.predict(input_example)

Out[9]: 0 409971.53125
Name: Price_p, dtype: float64

In [7]:
pipeline.avm(input_example)

Out[10]:

,unit_index,avm,avm_lower,avm_upper,conf,ts,latest_production_version,latest_staging_version
0,15289924,409972.0,374310.0,500281.0,0.779718,1625587451.060274,29,NaN


In [8]:
from pandas_profiling import ProfileReport

profile = ProfileReport(X_test.sample(500), title="Pandas Profiling Report", 
                        explorative=True
                       )
profile.to_widgets()

In [9]:

def _plot_accuracy_by_price_range(y_test, y_stacked, units):
    model_name='avm'

    df_acc = pd.DataFrame({'y_test': y_test, 'preds': y_stacked, 'unit_indx': units}, columns=['y_test', 'preds', 'unit_indx'])
    df_acc['error'] = abs((df_acc['preds'] - df_acc['y_test']) / df_acc['y_test'])
    df_acc['percent_error'] = (df_acc['preds'] - df_acc['y_test']) / df_acc['y_test']

    ctt = []
    try: 
        if len(df_acc[df_acc['y_test']<=50000]) > 5: ctt.append('df_0')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=50000) & (df_acc['y_test']<=100000)]) > 5: ctt.append('df_5')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=100000) & (df_acc['y_test']<=150000)]) > 5: ctt.append('df_10')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=150000) & (df_acc['y_test']<=200000)]) > 5: ctt.append('df_15')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=200000) & (df_acc['y_test']<=250000)]) > 5: ctt.append('df_20')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=250000) & (df_acc['y_test']<=300000)]) > 5: ctt.append('df_25')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=300000) & (df_acc['y_test']<=350000)]) > 5: ctt.append('df_30')
    except: pass
    try: 
        if len(df_acc[(df_acc['y_test']>=350000) & (df_acc['y_test']<=400000)]) > 5: ctt.append('df_35')
    except: pass
    try: 
        if len(df_acc[df_acc['y_test']>=400000]) > 5: ctt.append('df_40')
    except: pass

    def troubleshoot(ctt, df_acc):

        acc_2 = len(ctt)*[0]
        acc_5 = len(ctt)*[0]
        acc_10 = len(ctt)*[0]
        acc_20 = len(ctt)*[0]

        x = 0
        if 'df_0' in ctt:
            try:
                df_0 = df_acc[df_acc['y_test']<=50000]
                acc_2[x] = len(df_0[df_0['error']<0.02])/len(df_0)
                acc_5[x] = len(df_0[df_0['error']<0.05])/len(df_0)
                acc_10[x] = len(df_0[df_0['error']<0.1])/len(df_0)
                acc_20[x] = len(df_0[df_0['error']<0.2])/len(df_0)
                x += 1
            except: 
                ctt.pop('df_0')
                troubleshoot(ctt, df_acc)

        if 'df_5' in ctt:
            try:
                df_5 = df_acc[(df_acc['y_test']>=50000) & (df_acc['y_test']<=100000)]
                acc_2[x] = len(df_5[df_5['error']<0.02])/len(df_5)
                acc_5[x] = len(df_5[df_5['error']<0.05])/len(df_5)
                acc_10[x] = len(df_5[df_5['error']<0.10])/len(df_5)
                acc_20[x] = len(df_5[df_5['error']<0.2])/len(df_5)
                x += 1
            except: 
                ctt.pop('df_5')
                troubleshoot(ctt, df_acc)

        if 'df_10' in ctt:
            try:
                df_10 = df_acc[(df_acc['y_test']>=100000) & (df_acc['y_test']<=150000)]
                acc_2[x] = len(df_10[df_10['error']<0.02])/len(df_10)
                acc_5[x] = len(df_10[df_10['error']<0.05])/len(df_10)
                acc_10[x] = len(df_10[df_10['error']<0.10])/len(df_10)
                acc_20[x] = len(df_10[df_10['error']<0.2])/len(df_10)
                x += 1
            except: 
                ctt.pop('df_10')
                troubleshoot(ctt, df_acc)

        if 'df_10' in ctt:
            try:
                df_15 = df_acc[(df_acc['y_test']>=150000) & (df_acc['y_test']<=200000)]
                acc_2[x] = len(df_15[df_15['error']<0.02])/len(df_15)
                acc_5[x] = len(df_15[df_15['error']<0.05])/len(df_15)
                acc_10[x] = len(df_15[df_15['error']<0.1])/len(df_15)
                acc_20[x] = len(df_15[df_15['error']<0.2])/len(df_15)
                x += 1
            except: 
                ctt.pop('df_10')
                troubleshoot(ctt, df_acc)

        if 'df_20' in ctt:
            try:
                df_20 = df_acc[(df_acc['y_test']>=200000) & (df_acc['y_test']<=250000)]
                acc_2[x] = len(df_20[df_20['error']<0.02])/len(df_20)
                acc_5[x] = len(df_20[df_20['error']<0.05])/len(df_20)
                acc_10[x] = len(df_20[df_20['error']<0.1])/len(df_20)
                acc_20[x] = len(df_20[df_20['error']<0.2])/len(df_20)
                x += 1
            except: 
                ctt.pop('df_20')
                troubleshoot(ctt, df_acc)

        if 'df_25' in ctt:
            try:
                df_25 = df_acc[(df_acc['y_test']>=250000) & (df_acc['y_test']<=300000)]
                acc_2[x] = len(df_25[df_25['error']<0.02])/len(df_25)
                acc_5[x] = len(df_25[df_25['error']<0.05])/len(df_25)
                acc_10[x] = len(df_25[df_25['error']<0.1])/len(df_25)
                acc_20[x] = len(df_25[df_25['error']<0.2])/len(df_25)
                x += 1
            except: 
                ctt.pop('df_25')
                troubleshoot(ctt, df_acc)

        if 'df_30' in ctt:
            try:
                df_30 = df_acc[(df_acc['y_test']>=300000) & (df_acc['y_test']<=350000)]
                acc_2[x] = len(df_30[df_30['error']<0.02])/len(df_30)
                acc_5[x] = len(df_30[df_30['error']<0.05])/len(df_30)
                acc_10[x] = len(df_30[df_30['error']<0.1])/len(df_30)
                acc_20[x] = len(df_30[df_30['error']<0.2])/len(df_30)
                x += 1
            except: 
                ctt.pop('df_30')
                troubleshoot(ctt, df_acc)

        if 'df_35' in ctt:
            try:
                df_35 = df_acc[(df_acc['y_test']>=350000) & (df_acc['y_test']<=400000)]
                acc_2[x] = len(df_35[df_35['error']<0.02])/len(df_35)
                acc_5[x] = len(df_35[df_35['error']<0.05])/len(df_35)
                acc_10[x] = len(df_35[df_35['error']<0.1])/len(df_35)
                acc_20[x] = len(df_35[df_35['error']<0.2])/len(df_35)
                x += 1
            except: 
                ctt.pop('df_35')
                troubleshoot(ctt, df_acc)

        if 'df_40' in ctt:
            try:
                df_40 = df_acc[df_acc['y_test']>=400000]
                acc_2[x] = len(df_40[df_40['error']<0.02])/len(df_40)
                acc_5[x] = len(df_40[df_40['error']<0.05])/len(df_40)
                acc_10[x] = len(df_40[df_40['error']<0.1])/len(df_40)
                acc_20[x] = len(df_40[df_40['error']<0.2])/len(df_40)
                x += 1
            except: 
                ctt.pop('df_40')
                troubleshoot(ctt, df_acc)

        return acc_2, acc_5, acc_10, acc_20, ctt

    acc_2, acc_5, acc_10, acc_20, ctt = troubleshoot(ctt, df_acc)

    acc = pd.DataFrame(
        {'acc_2': acc_2,
         'acc_5': acc_5,
         'acc_10': acc_10,
         'acc_20': acc_20,
        })

    a = []
    if 'df_0' in ctt:
        a.append('<50k')
    if 'df_5' in ctt:
        a.append('50k-100k')
    if 'df_10' in ctt:
        a.append('100k-150k')
    if 'df_15' in ctt:
        a.append('150k-200k')
    if 'df_20' in ctt:
        a.append('200k-250k')
    if 'df_25' in ctt:
        a.append('250k-300k')
    if 'df_30' in ctt:
        a.append('300k-350k')
    if 'df_35' in ctt:
        a.append('350k-400k')
    if 'df_40' in ctt:
        a.append('>400k')

    acc['value bands'] = a

    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize = (20,10))
    #sns.catplot(x="acc_5", y="percentile", data=acc,ax=ax)
    ax.scatter(acc['acc_2'], acc['value bands'],label='err +-2%', s=80)
    ax.scatter(acc['acc_5'], acc['value bands'],label='err +-5%', s=80)
    ax.scatter(acc['acc_10'], acc['value bands'],label='err +-10%', s=80)
    ax.scatter(acc['acc_20'], acc['value bands'],label='err +-20%', s=80)

    for i in range(0, len(ctt)):
        plt.plot([acc['acc_2'][i], acc['acc_10'][i]], [[i]*5,[i]*5], 'grey')
        plt.plot([acc['acc_5'][i], acc['acc_20'][i]], [[i]*5,[i]*5], 'grey')

    plt.legend(fontsize=15)
    ax.set_xlim(0, 1)
    ax.set_xticklabels([0,20,40,60,80,100], rotation=0, fontsize=15)
    ax.set_yticklabels(acc['value bands'], rotation=0, fontsize=15)
    ax.set_xlabel('Percent of valuations within +-5%, +-20%')
    ax.set_ylabel('Price range of properties £ Pounds')
    plt.title('Accuracy by Value ' + model_name,fontsize=20)
    plt.show()
    imname = 'Accuracy by Price Range.png'
    plt.savefig(imname)
    print('end plot_by_price_range...')
    
    X_misvalued_abs = df_acc[df_acc['error']< 0.2]
    X_misvalued_low = df_acc[df_acc['percent_error'] < -0.2]
    X_misvalued_high = df_acc[df_acc['percent_error'] > 0.2]
    return X_misvalued_low, X_misvalued_high, X_misvalued_abs, df_acc
    

In [10]:
y_true = y_test['Price_p']
y_predicted = pipeline.predict(X_test)
units = X_test['unit_indx']

In [11]:
import matplotlib.pyplot as plt
_plot_accuracy_by_price_range(y_true, y_predicted, units)

end plot_by_price_range...
Out[30]: ( y_test preds unit_indx error percent_error
 13 183000.0 137485.781250 230275 0.248712 -0.248712
 19 300000.0 237360.218750 281918 0.208799 -0.208799
 25 244000.0 186228.703125 49306 0.236768 -0.236768
 31 126500.0 94390.664062 236586 0.253829 -0.253829
 40 432500.0 315312.906250 362602 0.270953 -0.270953
 ... ... ... ... ... ...
 109731 288000.0 199775.812500 377379 0.306334 -0.306334
 109758 350000.0 233671.515625 468864 0.332367 -0.332367
 109761 380000.0 303866.187500 35260 0.200352 -0.200352
 109802 330000.0 246918.828125 408741 0.251761 -0.251761
 109805 245000.0 177200.718750 2780 0.276732 -0.276732
 
 [8263 rows x 5 columns],
 y_test preds unit_indx error percent_error
 9 300000.0 365522.656250 781939 0.218409 0.218409
 22 100000.0 183882.421875 590087 0.838824 0.838824
 30 235000.0 349532.375000 836430 0.487372 0.487372
 32 130000.0 179247.500000 695346 0.378827 0.378827
 36 55000.0 72410.148438 851710 0.316548 0.316548
 ... ... ... ... ... ...
 109772 70000.0 122241.398438 754735 0.746306 0.746306
 109783 48000.0 59227.214844 195505 0.233900 0.233900
 109793 457000.0 562277.875000 572125 0.230367 0.230367
 109809 71500.0 104751.359375 434354 0.465054 0.465054
 109811 109500.0 194384.609375 54659 0.775202 0.775202
 
 [17058 rows x 5 columns],
 y_test preds unit_indx error percent_error
 0 697000.0 587934.687500 825917 0.156478 -0.156478
 1 357000.0 323038.531250 580808 0.095130 -0.095130
 2 337000.0 390568.718750 611607 0.158958 0.158958
 3 272000.0 227666.187500 763809 0.162992 -0.162992
 4 110000.0 104053.992188 347476 0.054055 -0.054055
 ... ... ... ... ... ...
 109807 166000.0 183119.968750 348631 0.103132 0.103132
 109808 255000.0 254291.343750 275956 0.002779 -0.002779
 109810 629000.0 509994.156250 141129 0.189198 -0.189198
 109812 420000.0 420398.312500 192470 0.000948 0.000948
 109813 110000.0 127456.585938 834679 0.158696 0.158696
 
 [84493 rows x 5 columns],
 y_test preds unit_indx error percent_error
 0 697000.0 587934.687500 825917 0.156478 -0.156478
 1 357000.0 323038.531250 580808 0.095130 -0.095130
 2 337000.0 390568.718750 611607 0.158958 0.158958
 3 272000.0 227666.187500 763809 0.162992 -0.162992
 4 110000.0 104053.992188 347476 0.054055 -0.054055
 ... ... ... ... ... ...
 109809 71500.0 104751.359375 434354 0.465054 0.465054
 109810 629000.0 509994.156250 141129 0.189198 -0.189198
 109811 109500.0 194384.609375 54659 0.775202 0.775202
 109812 420000.0 420398.312500 192470 0.000948 0.000948
 109813 110000.0 127456.585938 834679 0.158696 0.158696
 
 [109814 rows x 5 columns])